In [ ]:
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
train = pd.read_csv('train.csv')
dd = pd.read_csv('data_dictionary.csv')
test = pd.read_csv('test.csv')

testF = test.columns
trainF = train.columns
bothF = list(set(testF) & set(trainF))
missinF = list(set(trainF)-set(testF))
len(bothF),len(testF), len(trainF)

In [ ]:
def transform(df):
    df = df.loc[~df.sii.isna()]
    seasons = [s for s in df.columns if 'Season' in s]
    for s in seasons:
        df['isSameSeason'+s] = (df['Basic_Demos-Enroll_Season'] == df[s]).astype(np.int8)
    
    df = df.loc[df['Physical-Weight'] > 0]
    df['BIA-BIA_Fat'] = df['BIA-BIA_Fat'].clip(0,100)

    df['activityScore'] = np.where(pd.isna(df['PAQ_A-PAQ_A_Total']) & ~pd.isna(df['PAQ_C-PAQ_C_Total']), df['PAQ_C-PAQ_C_Total'],  # If a is NaN and b is not NaN, use b
                np.where(~pd.isna(df['PAQ_A-PAQ_A_Total']) & pd.isna(df['PAQ_C-PAQ_C_Total']), df['PAQ_A-PAQ_A_Total'],  # If a is not NaN and b is NaN, use a
                np.where(df['Basic_Demos-Age'] > 13, df['PAQ_A-PAQ_A_Total'], df['PAQ_C-PAQ_C_Total']))) 
    df['activityScoreSeason'] = np.where(pd.isna(df['PAQ_A-Season']) & ~pd.isna(df['PAQ_C-Season']), df['PAQ_C-Season'],  # If a is NaN and b is not NaN, use b
                np.where(~pd.isna(df['PAQ_A-Season']) & pd.isna(df['PAQ_C-Season']), df['PAQ_A-Season'],  # If a is not NaN and b is NaN, use a
                np.where(df['Basic_Demos-Age'] > 13, df['PAQ_A-Season'], df['PAQ_C-Season']))) 
    df=df.drop('Physical-Waist_Circumference',axis=1)
    return df

noNaDa = train.loc[~train.sii.isna()]
noNaDa = transform(noNaDa)

In [ ]:
noNaDa.loc[noNaDa['BIA-BIA_BMC']<00]#.max()

In [ ]:
pd.set_option('display.max_rows',None)
dd

In [ ]:
General: we have multiple physical tests:
- basics: general stats
- physical: bmi, height, weight,..
- fitness_endurance: on treadmil
- FitnessGram Child: lifting tests
- Bio-electric Impedance, fat & water mass,..
- Physical Activity Questionnaire: adult vs child 
- Sleep Disturbance Scale
- Internet Use
- Parent-Child Internet Addiction Test (only in train)


semester: very uniform, no info to target
age: more younger childs, not gaussian, linear trend of mean, the older the more likely to have issue
sex: more men than women, men are more likely to have issue, categorical feature
     women seem to be a bit older when they have issues, linear trend is slightly different (combine features)


First strategy:
- handle everything as categorical -> quantile transformation of data (less impact of outliers)

# quantile encoding of numerical values

In [ ]:
from sklearn.preprocessing import QuantileTransformer

In [ ]:
numerical_columns=[]
for f in train.select_dtypes(np.float64).columns:
    if len(train[f].unique())>30:
        numerical_columns.append(f)
numerical_columns

In [ ]:
def addEncodings(train):
    numerical_columns=[]
    for f in train.select_dtypes(np.float64).columns:
        if len(train[f].unique())>30:
            numerical_columns.append(f) 

    normalDistF = [f+'normalD' for f in numerical_columns]
    unifDistF = [f+'unifD' for f in numerical_columns]
    qtUnif = QuantileTransformer(n_quantiles=1000, output_distribution='uniform', random_state=42)  
    qtNorm = QuantileTransformer(n_quantiles=1000, output_distribution='normal', random_state=42)  

    train[unifDistF] = qtUnif.fit_transform(train[numerical_columns])
    train[unifDistF] = qtNorm.fit_transform(train[numerical_columns])

    return train, qtUnif, qtNorm

In [ ]:
numerical_columns=[]
for f in train.select_dtypes(np.float64).columns:
    if len(train[f].unique())>30:
        numerical_columns.append(f)

normalDistF = [f+'normalD' for f in numerical_columns]
unifDistF = [f+'unifD' for f in numerical_columns]
qt = QuantileTransformer(n_quantiles=1000, output_distribution='uniform', random_state=42)

# Fit and transform the data
trainQ = train.copy()
trainQ[numerical_columns] = qt.fit_transform(train[numerical_columns])

In [ ]:
pd.set_option('display.max_rows',80)
trainQ.describe().T

# handle missing values

In [ ]:
noNaDa.describe().T

In [ ]:
pd.set_option('display.max_columns',90)
noNaDa.loc[(noNaDa['PAQ_A-PAQ_A_Total'].isna() == False) & (noNaDa['PAQ_C-PAQ_C_Total'].isna() == False)]

In [ ]:

noNaDa['activityScore'] = np.where(pd.isna(noNaDa['PAQ_A-PAQ_A_Total']) & ~pd.isna(noNaDa['PAQ_C-PAQ_C_Total']), noNaDa['PAQ_C-PAQ_C_Total'],  # If a is NaN and b is not NaN, use b
                np.where(~pd.isna(noNaDa['PAQ_A-PAQ_A_Total']) & pd.isna(noNaDa['PAQ_C-PAQ_C_Total']), noNaDa['PAQ_A-PAQ_A_Total'],  # If a is not NaN and b is NaN, use a
                np.where(noNaDa['Basic_Demos-Age'] > 13, noNaDa['PAQ_A-PAQ_A_Total'], noNaDa['PAQ_C-PAQ_C_Total']))) 

In [ ]:
noNaDa[['PAQ_A-PAQ_A_Total','PAQ_C-PAQ_C_Total','activityScore']]

# target

- na for a bunch of values
- discrete in 0,1,2,3
- frequency is going down linearly from 0 (most frequent) to 3  (least frequent)

In [ ]:
noNaDa = train.loc[~train.sii.isna()]
noNaDa = noNaDa.loc[noNaDa['Physical-Weight'] > 0]

In [ ]:
pd.set_option('display.max_rows',71)
noNaDa.describe().T

In [ ]:
test.describe().T

In [ ]:
noNaDa.sii.hist(bins=100)

# features

In [ ]:
dtype_dict = dict(noNaDa.dtypes)
for column, dtype in dtype_dict.items():
    print(f"{column}: {dtype}")

In [ ]:
feat = ['id', 'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
       'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
       'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
       'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
       'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
       'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
       'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
       'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
       'PAQ_C-PAQ_C_Total', 'PCIAT-Season', 'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02',
       'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06',
       'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10',
       'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14',
       'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18',
       'PCIAT-PCIAT_19', 'PCIAT-PCIAT_20', 'PCIAT-PCIAT_Total', 'SDS-Season',
       'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
       'PreInt_EduHx-computerinternet_hoursday', 'sii']
#noNaDa[f].value_counts(), 

f = 'BIA-BIA_ICW'# feat[5]
def eval(f, noNaDa):
    print(f)
    display(noNaDa[f].isna().sum())
    #noNaDa[f].hist(bins=100)
    #grouped = noNaDa.groupby('sii')[f].value_counts().unstack().fillna(0)
    #grouped = noNaDa.groupby('sii')[f]
    min_val = noNaDa[f].min()  # Minimum value across all groups
    max_val = noNaDa[f].max()  # Maximum value across all groups
    print(min_val, max_val)
    # Define the number of bins and calculate bin edges
    num_bins = 100
    bins = np.linspace(min_val, max_val, num_bins + 1)
    fig, ax = plt.subplots(figsize=(10, 5))
    for i in noNaDa.sii.unique():
        ax.hist(noNaDa.loc[noNaDa.sii==i][f], bins=bins, alpha=0.2, label=i)
    plt.title(f'Stacked Bar Plot of {f} grouped by a')
    plt.xlabel('a')
    plt.ylabel('Count')
    plt.legend(title=f, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
    if noNaDa[f].dtype == np.float64:
        sns.violinplot(x=noNaDa['sii'],y=noNaDa[f])
    else:
        display(pd.crosstab(noNaDa['sii'],noNaDa[f]).style.background_gradient(cmap='viridis'))
        display(pd.crosstab(noNaDa['sii'],noNaDa[f], normalize='index').style.background_gradient(cmap='viridis', vmin=0, vmax=1))
        display(pd.crosstab(noNaDa['sii'],noNaDa[f], normalize='columns').style.background_gradient(cmap='viridis', vmin=0, vmax=1))
eval(f,noNaDa)

In [ ]:
grouped = noNaDa.groupby('sii')[f]
grouped.hist()


In [ ]:
seasons = [s for s in trainF if 'Season' in s]
modSFeat = []
for s in seasons:
    #noNaDa['isSameSeason'+s] = (noNaDa['Basic_Demos-Enroll_Season'] == noNaDa[s]).astype(np.int8)
    modSFeat.append('isSameSeason'+s)
    print(noNaDa['isSameSeason'+s].value_counts())

## bio impedance measurements


In [ ]:
zScore = (noNaDa['BIA-BIA_ICW'] - noNaDa['BIA-BIA_ICW'].mean())/noNaDa['BIA-BIA_ICW'].std()
noNaDa['BIA-BIA_ICW'].describe()
zScore.dropna().sort_values()

In [ ]:
pd.set_option('display.max_rows',81)
pd.set_option('display.max_columns',101)
noNaDa.loc[noNaDa['BIA-BIA_ICW']>200]#['BIA-BIA_ICW']

In [ ]:
test#.loc[test['BIA-BIA_ICW']>50]

## enrolled vs participated season

## sex and age relation

In [ ]:
men = noNaDa.loc[noNaDa['Basic_Demos-Sex'] == 0]
eval(f,men)
women = noNaDa.loc[noNaDa['Basic_Demos-Sex'] == 1]
eval(f,women)

# create reports for each feature

In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from io import BytesIO
from tqdm import tqdm

In [ ]:
def eval_to_pdf(f, noNaDa, pdf_filename):
    c = canvas.Canvas(pdf_filename, pagesize=A4)
    width, height = A4

    # Title
    c.setFont("Helvetica-Bold", 16)
    c.drawString(50, height - 50, f"Analysis of {f}")

    # Missing values
    c.setFont("Helvetica", 12)
    missing_values = noNaDa[f].isna().sum()
    c.drawString(50, height - 80, f"Missing values: {missing_values}")

    # hist
    plt.figure(figsize=(10, 5))
    noNaDa[f].hist(bins=30)
    plt.title(f'Histogram of {f}')
    plt.xlabel(f)
    plt.ylabel('Count')
    plt.tight_layout()
    
    img_buffer2 = BytesIO()
    plt.savefig(img_buffer2, format='png')
    img_buffer2.seek(0)
    c.drawImage(ImageReader(img_buffer2), 50, height - 350, width=500, height=250)
    plt.close()

    c.showPage()
    # Violin plot or crosstab
    if noNaDa[f].dtype == np.float64 and len(noNaDa[f].unique())>20:
        zScore =(noNaDa[f]-noNaDa[f].mean())/(noNaDa[f].std())
        # hist
        noOut = noNaDa#.loc[abs(zScore)<5]
        c.drawString(50, height - 810, f"outliers values: {noNaDa.loc[abs(zScore)>5].shape[0]}")

        fig, (ax1, ax2,ax3,ax4,ax5) = plt.subplots(5, 1, figsize=(10, 10))
        # Violin plot
        sns.violinplot(x=noOut['sii'], y=noOut[f], ax=ax1)
        ax1.set_title(f'Violin Plot of {f} by sii')
        
        # Correlation matrix
        #corr_matrix = noNaDa[['sii', f]].corr(method='spearman')
        #sns.heatmap(corr_matrix, annot=True, cmap='RdBu', ax=ax2, vmin=-1,vmax=1)
        #ax2.set_title('Correlation Matrix')
        #noOut['siiJitter'] = noOut['sii'] + np.random.random()*.8 - 0.4
        noOut=noOut.assign(**{'siiJitter': noOut['sii'] + np.random.random(len(noOut))*.8 -.4})
        noOut.plot.scatter(x='siiJitter', y=f, ax=ax2, alpha=0.1)
        ax2.set_xlabel('sii jitter')
        ax2.set_ylabel(f)
        noOut[f].hist(bins=30, ax=ax3)
        noOut.plot.hexbin(x='siiJitter', y=f, alpha=1, gridsize=10, ax=ax4)

        min_val = noOut[f].min()  # Minimum value across all groups
        max_val = noOut[f].max()  # Maximum value across all groups
        #print(min_val, max_val)
        # Define the number of bins and calculate bin edges
        num_bins = 100
        bins = np.linspace(min_val, max_val, num_bins + 1)
        for i in noOut.sii.unique():
            ax5.hist(noOut.loc[noOut.sii==i][f], bins=bins, alpha=0.2, label=i)
        ax5.set_xlabel(f)
        ax5.set_ylabel('frequency')
        plt.legend()
        plt.tight_layout()
        
        img_buffer = BytesIO()
        plt.savefig(img_buffer, format='png')
        img_buffer.seek(0)
        c.drawImage(ImageReader(img_buffer), 50, height - 800, width=500, height=800)
        plt.close()

        # 'Basic_Demos-Age', 'Basic_Demos-Sex', 'Physical-Height', 'Physical-Weight',
        c.showPage()
        fig2, (ax1, ax2,ax3,ax4,ax5,ax6) = plt.subplots(6, 1, figsize=(10, 10))
        noOut=noOut.assign(**{'ageJitter': noOut['Basic_Demos-Age'] + np.random.random(len(noOut))*.8 -.4})
        #noOut.plot.hexbin(x='Basic_Demos-Age', y=f, alpha=1, gridsize=20, ax=ax1)
        noOut.plot.scatter(x='ageJitter', y=f, alpha=0.1, ax=ax1)
        sns.violinplot(x=noOut['Basic_Demos-Age'], y=noOut[f], ax=ax2)
        noOut=noOut.assign(**{'sexJitter': noOut['Basic_Demos-Sex'] + np.random.random(len(noOut))*.8 -.4})
        #noOut.plot.hexbin(x='Basic_Demos-Sex', y=f, alpha=1, gridsize=20, ax=ax2)
        noOut.plot.scatter(x='sexJitter', y=f, alpha=0.1, ax=ax3)
        sns.violinplot(x=noOut['Basic_Demos-Sex'], y=noOut[f], ax=ax4)
        noOut.plot.scatter(x='Physical-Height', y=f, alpha=0.1, ax=ax5)
        noOut.plot.scatter(x='Physical-Weight', y=f, alpha=0.1, ax=ax6)
        plt.legend()
        plt.tight_layout()
        
        img_buffer = BytesIO()
        plt.savefig(img_buffer, format='png')
        img_buffer.seek(0)
        c.drawImage(ImageReader(img_buffer), 50, height - 800, width=500, height=800)
        plt.close()
    else:
        # Bar plot
        grouped = noNaDa.groupby(f)['sii'].value_counts().unstack().fillna(0)
        fig, (ax1,ax2) = plt.subplots(2, 1, figsize=(10, 5))
        grouped.plot(kind='bar', stacked=False, ax=ax1)
        grouped.plot(kind='bar', stacked=True, ax=ax2)
        plt.title(f'Bar Plot of {f} grouped by sii')
        plt.xlabel(f)
        plt.ylabel('Count')
        plt.legend(title='sii', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()

        img_buffer = BytesIO()
        plt.savefig(img_buffer, format='png')
        img_buffer.seek(0)
        c.drawImage(ImageReader(img_buffer), 50, height - 400, width=500, height=250)
        plt.close()
        c.showPage()
        # Create crosstabs
        crosstab = pd.crosstab(noNaDa['sii'], noNaDa[f])
        crosstab_norm_index = pd.crosstab(noNaDa['sii'], noNaDa[f], normalize='index')
        crosstab_norm_columns = pd.crosstab(noNaDa['sii'], noNaDa[f], normalize='columns')

        # Function to create and save heatmap
        def create_heatmap(data, title,normalized=True):
            plt.figure(figsize=(10, 6))
            if normalized:
                sns.heatmap(data, annot=True, cmap='viridis', fmt='.2f', vmin=0, vmax=1)
            else:
                sns.heatmap(data, annot=True, cmap='viridis', fmt='.0f')
            plt.title(title)
            plt.tight_layout()
            
            img_buffer = BytesIO()
            plt.savefig(img_buffer, format='png')
            img_buffer.seek(0)
            return img_buffer

        # Create heatmaps
        heatmap_regular = create_heatmap(crosstab, 'Crosstab', False)
        heatmap_norm_index = create_heatmap(crosstab_norm_index, 'Crosstab (normalized by index)')
        heatmap_norm_columns = create_heatmap(crosstab_norm_columns, 'Crosstab (normalized by columns)')

        # Add heatmaps to PDF
        c.drawImage(ImageReader(heatmap_regular), 50, height - 700, width=500, height=250)
        c.showPage()  # New page for normalized crosstabs
        c.drawImage(ImageReader(heatmap_norm_index), 50, height - 300, width=500, height=250)
        c.drawImage(ImageReader(heatmap_norm_columns), 50, height - 600, width=500, height=250)

    c.showPage()
    c.save()

# Usage
if 1:
    for i,f in enumerate(tqdm(trainF)):
        if f == 'ii' or f =='id':
            continue
        eval_to_pdf(f, trainQ, 'reportsQuantile/'+str(i)+'output_report'+f+'.pdf')
else:
    f = 'CGAS-CGAS_Score'
    #f = 'BIA-BIA_ICW'
    #eval_to_pdf(f, noNaDa, 'output_report'+f+'.pdf')